<img style="float: left;" src="../../images/logo.png" width="400"/>

The codes below is inspired from the official GFED website, see [here](https://www.geo.vu.nl/~gwerf/GFED/GFED4/ancill/code/get_GFED4s_CO_emissions.py) for original source.

In this example we will calculate annual **CO emissions** for the 14 GFED basis regions over 1997-2014.
The distributed version of GFED4 stopped in 2014, so that we could not look at el pont de Vilomara here. [GFED5](https://essd.copernicus.org/articles/15/5227/2023/essd-15-5227-2023-discussion.html) is also available but stopped in 2020. 

## 1. To go though the data

In [ ]:
### 1.1 access the data

First you need to get the GFED4.1s data available in this notebook.
- data is available directly [dropboxlink](https://www.dropbox.com/scl/fi/fgi1m3nnk436k9a0bhdpi/GFED4.1s.tar.gz?rlkey=vlaobhovce3v3lhr6uhfk4fd5&st=989ruqnp&dl=0) or in open access on [here](https://daac.ornl.gov/VEGETATION/guides/fire_emissions_v4_R1.html)
- or if you are on andromeda the data are directly accessible here `/data/IMFSE/FBM_unit05/Emission/GFED4.1s`

The you need to set the variable directory to point to where you have the data available.

In [4]:
directory    = '/data/IMFSE/FBM_unit05/Emission/GFED4.1s/data/'

In GFED4.1s biomass burning dry matter (DM) emissions are provided in `kg of DM/m2/month` in the yearly file `GFED4.1s_xxxx.hdf5`. This corresponds to the mass of burnt fuel.

### 1.2. import library and set global variables

In [6]:
# import library
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt

In [17]:
#definition of variables
months       = '01','02','03','04','05','06','07','08','09','10','11','12'
sources      = 'SAVA','BORF','TEMF','DEFO','PEAT','AGRI'
Regions=['BONA', 'TENA', 'CEAM', 'NHSA', 'SHSA', 'EURO', 'MIDE', 'NHAF' ,'SHAF', 'BOAS', 'TEAS', 'SEAS', 'EQAS', 'AUST', 'Global' ]
#Boreal North America (BONA), 
#Temperate North America (TENA), 
#Central America (CEAM), 
#Northern Hemisphere South America (NHSA), 
#Southern Hemisphere South America (SHSA), 
#Europe (EURO), 
#Middle East (MIDE), 
#Northern Hemisphere Africa (NHAF), 
#Southern Hemisphere Africa (SHAF), 
#Boreal Asia (BOAS), 
#Central Asia (CEAS), 
#Southeast Asia (SEAS), 
#Equatorial Asia (EQAS) 
#Australia and New Zealand (AUST).
    
start_year = 1997
end_year   = 2014

### 1.3. Read in emission factors

In [18]:
species = [] # names of the different gas and aerosol species
EFs     = np.zeros((41, 6)) # 41 species, 6 sources

data_EF = pd.read_csv(directory+'/GFED4_Emission_Factors_Summary.csv', header=10)

data_EF # to print Emssion fator


,Species,SAVA,BORF,TEMF,DEFO,PEAT,AGRI
0,DM,1000.0000,1000.00000,1000.0000,1000.00000,1000.00000,1000.0000
1,C,488.2730,464.98900,489.4160,491.75100,570.05500,480.3520
2,CO2,1686.0000,1489.00000,1647.0000,1643.00000,1703.00000,1585.0000
3,CO,63.0000,127.00000,88.0000,93.00000,210.00000,102.0000
4,CH4,1.9400,5.96000,3.3600,5.07000,20.80000,5.8200
5,NMHC,3.4000,8.40000,8.4000,1.70000,1.70000,9.9000
6,H2,1.7000,2.03000,2.0300,3.36000,3.36000,2.5900
7,Nox,3.9000,0.90000,1.9200,2.55000,1.00000,3.1100
8,N2O,0.2000,0.41000,0.1600,0.20000,0.20000,0.1000
9,PM2.5,7.1700,15.30000,12.9000,9.10000,9.10000,6.2600


In [19]:
# we are interested in CO for this example (4th row, so index=3):

EF_CO = np.array(data_EF.iloc[3])[1:]

### 1.4. Make table with summed DM emissions for each region and year

The code below load the burnt dry mass computed per month from the burn area product.  
It is available at global scale, monthly, and at 0.25 degree spatial resolution from mid-1995 through 2014.   
 
The data were derived by combining 500-m **MODIS** burned area maps with active fire data from the Tropical Rainfall Measuring Mission (**TRMM**) Visible and Infrared Scanner (**VIRS**) and the Along-Track Scanning Radiometer (**ATSR**) family of sensors. For additional information, refer to [Giglio et al., 2013](https://doi.org/10.1002/jgrg.20042).

In [23]:
#we define a function here so that it can be used easily fro other fire emission than CO

def computeEmissionRegionYear(EF_XX):
    XX_table = np.zeros((15, end_year - start_year + 1)) # region, year
    years =[]

    for year in range(start_year, end_year+1):
        string = directory+'/GFED4.1s_'+str(year)+'.hdf5'
        f = h5py.File(string, 'r')
        print(year)
        years.append(year)
    
        if year == start_year: # these are time invariable    
            basis_regions = f['/ancill/basis_regions'][:]
            grid_area     = f['/ancill/grid_cell_area'][:]


        XX_emissions = np.zeros((720, 1440))
        for month in range(12):
            # read in DM emissions
            string = '/emissions/'+months[month]+'/DM'
            DM_emissions = f[string][:]
            for source in range(6): # 6 source of fire emission are: SAVA,BORF,TEMF,DEFO,PEAT,AGRI
                # read in the fractional contribution of each source
                string = '/emissions/'+months[month]+'/partitioning/DM_'+sources[source]
                contribution = f[string][:]
                # calculate XX emissions as the product of DM emissions (kg DM per 
                # m2 per month), the fraction the specific source contributes to 
                # this (unitless), and the emission factor (g XX per kg DM burned)
                XX_emissions += DM_emissions * contribution * float(EF_XX[source])
    
    
        # fill table with total values for the globe (row 15) or basisregion (1-14)
        for region in range(15):
            if region == 14:
                mask = np.ones((720, 1440))
            else:
                mask = basis_regions == (region + 1)            
    
            XX_table[region, year-start_year] = np.sum(grid_area * mask * XX_emissions)
        

        
    # convert to Tg CO 
    XX_table = XX_table / 1E12
    
    #convert to dataframe
    XX_df = pd.DataFrame(XX_table, index=Regions, columns=years)

    return XX_df

In [25]:
CO_df = computeEmissionRegionYear(EF_CO)

1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014


In [26]:
#print result
CO_df

,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
BONA,6.712560,24.116295,10.048192,5.523846,3.036932,23.215209,14.536750,27.089686,18.681726,13.312017,10.285208,8.256071,10.767712,19.329582,13.153562,13.681917,27.170445,31.160299
TENA,1.827541,2.196653,3.452715,3.423350,1.894193,2.926240,2.772448,2.169272,2.456773,3.246291,4.738350,3.537972,2.427957,2.859341,3.250302,4.141402,2.706534,3.038820
CEAM,3.532858,30.003065,4.044757,9.256392,2.613084,3.509161,10.871073,2.594409,8.547834,4.665963,5.278977,4.548545,5.320212,3.229445,7.364683,2.701722,6.525271,2.411448
NHSA,5.348964,7.500564,3.996341,5.219141,5.679365,3.192058,9.330445,4.641932,3.470294,2.926189,6.267748,3.588066,3.929599,4.640624,1.831893,2.602856,2.963382,4.737330
SHSA,51.440661,69.584390,63.098720,29.131511,26.480714,48.626692,39.788552,59.851584,74.626636,34.441149,89.213917,29.921477,16.157629,92.454113,23.701130,45.892602,21.817676,31.740338
EURO,1.388813,1.522301,1.122716,3.520102,2.109920,0.897793,1.649581,1.324431,1.457506,1.347652,1.661915,1.546884,0.941453,0.921170,1.348563,2.038783,0.916038,0.881814
MIDE,0.164691,0.465797,0.292002,0.291255,0.268512,0.247295,0.307834,0.323859,0.299485,0.313289,0.410498,0.247084,0.440124,0.461558,0.393407,0.353472,0.341329,0.326163
NHAF,67.714618,89.885038,72.189567,77.614075,65.052657,68.420923,61.334357,57.115242,70.876513,56.347646,61.699561,61.596833,53.830626,49.408471,56.632084,50.787773,53.470833,50.419279
SHAF,82.169430,96.298947,81.862585,77.350403,104.793631,87.301255,91.276984,91.529023,96.214569,90.443127,84.654076,91.593500,91.240416,98.453321,94.045522,85.787163,93.444519,85.474370
BOAS,23.787223,73.892213,25.733273,28.712728,26.055807,42.291505,73.558103,10.762001,16.776803,32.954606,14.811624,45.916909,16.899277,21.919305,26.523630,56.832205,22.123178,33.203559


## 2. To go further

1. on the 18 years, which region is the principal source of CO emitted from burning vegetation?
2. Can you plot yearly evolution for each source?
3. Can you compute CO2 Emission? Is the spatial distribution per area the same?
4. Using the result from the last exercise of the introductionary python lecture (see [here](https://github.com/3dfirelab/python-intro-course/blob/master/06-pandasExcercise.ipynb)) can you calculate total carbone emission for you country over the 18 years.? 

## 3. To deliver